In [1]:
# Install required libraries
!pip install groq
!pip install streamlit
!pip install pyngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [2]:
%%writefile app.py
import os
import streamlit as st
from groq import Groq

# Retrieve the API key from the environment variable
GROQ_API_KEY = os.environ.get('GROQ_API_KEY')

# Initialize the Groq API client
client = Groq(api_key=GROQ_API_KEY)

# Define the product with base price and minimum acceptable price
class Product:
    def __init__(self, name, base_price, min_price):
        self.name = name
        self.base_price = base_price
        self.min_price = min_price
        self.current_offer = base_price

    def update_offer(self, user_offer):
        if user_offer >= self.min_price and user_offer < self.current_offer:
            self.current_offer = user_offer
            return True
        elif user_offer < self.min_price:
            # Can't accept offer below min price, make a counteroffer
            self.current_offer = max(self.min_price, self.current_offer - 50)
            return False
        else:
            # Offer is acceptable
            self.current_offer = user_offer
            return True

# Function to get response from Llama 3 via Groq API with error handling
def get_bot_response(messages, model="llama3-8b-8192"):
    try:
        chat_completion = client.chat.completions.create(
            messages=messages,
            model=model,
        )
        response = chat_completion.choices[0].message.content.strip()
        return response
    except Exception as e:
        st.error(f"An error occurred while getting bot response: {e}")
        return "I'm sorry, but I'm having trouble responding at the moment."

def analyze_sentiment(user_message):
    # Use Llama 3 to perform sentiment analysis with error handling
    sentiment_prompt = f"Please analyze the sentiment of the following message and respond with one word (Positive, Negative, or Neutral):\n\n{user_message}"
    try:
        sentiment_response = client.chat.completions.create(
            messages=[
                {"role": "user", "content": sentiment_prompt}
            ],
            model="llama3-8b-8192",
        )
        sentiment = sentiment_response.choices[0].message.content.strip()
        return sentiment
    except Exception as e:
        st.error(f"An error occurred while analyzing sentiment: {e}")
        return "Neutral"  # Default to Neutral if there's an error

# Streamlit app
def main():
    st.title("Negotiation Chatbot")

    if 'messages' not in st.session_state:
        st.session_state.messages = [
            {"role": "system", "content": "You are a helpful assistant that negotiates product prices with customers."},
            {"role": "assistant", "content": "Hello! I'm here to help you with your purchase. What product are you interested in?"}
        ]

    if 'product' not in st.session_state:
        # Initialize the product
        st.session_state.product = Product("Laptop", base_price=1000, min_price=800)

    # Display conversation history
    for msg in st.session_state.messages[1:]:  # Skip the system message
        if msg["role"] == "user":
            st.write(f"**You:** {msg['content']}")
        elif msg["role"] == "assistant":
            st.write(f"**Bot:** {msg['content']}")

    # Use st.form to handle user input without needing rerun
    with st.form(key='chat_form', clear_on_submit=True):
        user_input = st.text_input("Type your message here:")
        submit_button = st.form_submit_button(label='Send')

    if submit_button and user_input:
        # Append user's message
        st.session_state.messages.append({"role": "user", "content": user_input})

        # Analyze sentiment
        sentiment = analyze_sentiment(user_input)
        st.write(f"Sentiment detected: {sentiment}")  # Display sentiment

        # Adjust pricing based on sentiment
        if sentiment.lower() == "positive":
            # Offer a better deal
            st.session_state.product.current_offer -= 25  # Reduce price by $25
        elif sentiment.lower() == "negative":
            # Be firmer on the price
            st.session_state.product.current_offer -= 10  # Be firmer on the price but offer slight discount

        # Simulate pricing logic
        try:
            # Attempt to extract an offer from the user's message
            user_offer = float(''.join(c for c in user_input if c.isdigit() or c == '.'))
            accepted = st.session_state.product.update_offer(user_offer)

            if accepted:
                bot_message = f"Great! We accept your offer of ${st.session_state.product.current_offer}."
                st.session_state.negotiation_over = True
            else:
                bot_message = f"How about we agree on ${st.session_state.product.current_offer}?"
        except ValueError:
            # If user input is not a price, get response from Llama 3
            bot_message = get_bot_response([
                {"role": "system", "content": "You are a sales assistant negotiating prices with customers."},
                {"role": "assistant", "content": st.session_state.messages[-2]['content']},
                {"role": "user", "content": user_input}
            ])

        # Append bot's message
        st.session_state.messages.append({"role": "assistant", "content": bot_message})

        # Display the bot's response
        st.markdown(f"**Bot:** {bot_message}")

if __name__ == '__main__':
    main()


Writing app.py


In [3]:
import os
import getpass
from google.colab import userdata
GROQ_API=userdata.get('GROQ_API')
NGROK_AUTH_TOKEN=userdata.get('NGROK_AUTH_TOKEN')

# Set the API key as an environment variable
os.environ['GROQ_API_KEY'] = GROQ_API

# Securely input your ngrok authtoken
#NGROK_AUTH_TOKEN = getpass.getpass('Please enter your ngrok authtoken: ')

# Set the ngrok authtoken as an environment variable
os.environ["NGROK_AUTH_TOKEN"] = NGROK_AUTH_TOKEN




In [4]:
# Start ngrok tunnel
from pyngrok import ngrok

# Kill any existing tunnels
ngrok.kill()
# Retrieve the API key from the environment variable
NGROK_AUTH_TOKEN = os.environ.get('NGROK_AUTH_TOKEN')
# Set the ngrok authtoken
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Start a new ngrok tunnel
public_url = ngrok.connect(addr='8501')
print(f"Streamlit app will be available at: {public_url}")


Streamlit app will be available at: NgrokTunnel: "https://0d24-34-90-249-115.ngrok-free.app" -> "http://localhost:8501"


In [5]:
# Run the Streamlit app
!streamlit run app.py --server.port 8501 >/dev/null 2>&1 &
